## Short-read Analysis

This notebooks produces basic statistics and figures for the short-read data for single-cell PBMCs: number of cells, nUMIs/cell, nGenes/cell, etc. Next, we subsample to a roughly equal number of reads per cell, recompute our statistics, and cluster the three datasets.

The subsampling portion of the notebook relies on [`bugzapper`](https://github.com/MethodsDev/bugzapper.git), a small Rust+Python library for parsing BAM files and counting up the barcode and UMIs. To reproduce the subsampling one will need to install this package and download the aligned BAM files. The package can installed with `pip install git+ssh://git@github.com/MethodsDev/bugzapper.git`.

Data is from the `230924_SL-EXC_0072_A2275VKLT3` flowcell and contains four samples:

  * 10x 3' PBMC
  * 10x 5' PBMC
  * Fluent PBMC (0.8x SPRI)
  * Fluent Barnyard (K562 + 3T3)

The data files for this notebook (the output from CellRanger and PIPseeker) are in a tarball: [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_matrices.tgz`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_matrices.tgz). Download is 2.1GB.

The pickle files (checkpoints to avoid computing anything) are in separate files:
  * [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_full_stats.pickle`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_full_stats.pickle)
  * [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_clustering_100k.pickle`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_clustering_100k.pickle)
  * [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_graphs_100k.pickle`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_graphs_100k.pickle)
  * [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_stats_100k.pickle`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_stats_100k.pickle)
  * [`gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/subsample_stats.pickle`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/notebook_checkpoints/subsample_stats.pickle)

These take up about 150M in total.

Output from CellRanger and PIPseeker:
  * [`gs://mdl-sc-isoform-2025-ms/pbmc_pipseq`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/pbmc_pipseq)
  * [`gs://mdl-sc-isoform-2025-ms/pbmc_10x_3p`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/pbmc_10x_3p)
  * [`gs://mdl-sc-isoform-2025-ms/pbmc_10x_5p`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/pbmc_10x_5p)
  * [`gs://mdl-sc-isoform-2025-ms/barnyard_pipseq`](https://console.cloud.google.com/storage/browser/mdl-sc-isoform-2025-ms/barnyard_pipseq)


### Imports

In [ ]:
import csv
import gzip
import pickle
from collections import defaultdict

from pathlib import Path

import numpy as np
import sparse

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D

import bugzapper

# a bunch of functions for single-cell analysis
# installed from www.github.com/methodsdev/isoscelles
import mdl.isoscelles.neighbors as nbrs
from mdl.isoscelles.gene_selection import fit_poisson
from mdl.isoscelles.io import read_mtx
from mdl.isoscelles.leiden import recursive_cluster, cluster_leaf_nodes, cluster_labels

# code from this paper repository
from mdl.sc_isoform_paper.constants import SHORTREAD_KEYS, SHORTREAD_NAMES, SAMPLE_COLORS
from mdl.sc_isoform_paper.graph_layout import layout_graph
from mdl.sc_isoform_paper.plots import plot_dists, plot_marker_dotplot

In [ ]:
# constant for formatting legend 
line_kwargs = dict(
    xdata=[0], ydata=[0], marker="o", linewidth=0, markeredgewidth=0, markersize=10
)

# use a seed to generate a consistent subsample
rng = np.random.default_rng(seed=sum(map(ord, "MDL")))


#### File I/O and preprocessing

The raw files are quite slow to process, and bulky to keep in RAM. It's convenient to convert them to sparse arrays and save as `.npz` files. With enough RAM it'd be okay to just store them in memory instead.

In [ ]:
# path to the short-read data (output from cellranger + pipseeker)
root_dir = Path.home()
data_path = root_dir / "data"
figure_path = root_dir / "202501_figures"

In [ ]:
# uncomment to download to local disk
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/shortread_matrices.tgz - | tar -C {data_path} -xzv
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/"shortread*.pickle" {data_path}/

In [ ]:
# convert the 10x mtx files
for fp in data_path.glob("10x*/outs/raw_feature_bc_matrix/matrix.mtx.gz"):
    out_fp = fp.parent.parent / f"{fp.parent.parent.parent.name}.npz"
    print(fp, out_fp)
    if out_fp.exists():
        print("File already exists")
        continue

    matrix = read_mtx(fp)
    sparse.save_npz(out_fp, matrix)

In [ ]:
# convert the PIPseq mtx files. We'll convert the barnyard data while we're here
for fp in data_path.glob("pipseq_*/raw_matrix/matrix.mtx.gz"):
    out_fp = fp.parent / f"{fp.parent.parent.name}.npz"
    print(fp, out_fp)
    if out_fp.exists():
        print("File already exists")
        continue

    matrix = read_mtx(fp)
    sparse.save_npz(out_fp, matrix)

### Loading feature and barcode lists

In [ ]:
bc_dict = dict()
feature_dict = dict()
gene_dict_i = dict()

for fp in data_path.glob("10x*/outs/raw_feature_bc_matrix/"):
    k = fp.parent.parent.name
    print(fp)
    with gzip.open(fp / "barcodes.tsv.gz", "rt") as fh:
        bc_dict[k] = [line.strip() for line in fh]

    with gzip.open(fp / "features.tsv.gz", "rt") as fh:
        feature_dict[k] = list(csv.reader(fh, delimiter="\t"))

    gene_dict_i[k] = {g[1]: i for i, g in enumerate(feature_dict[k])}

for fp in data_path.glob("pipseq*/raw_matrix/"):
    k = fp.parent.name
    print(fp)
    with gzip.open(fp / "barcodes.tsv.gz", "rt") as fh:
        bc_dict[k] = [line.strip() for line in fh]

    with gzip.open(fp / "features.tsv.gz", "rt") as fh:
        feature_dict[k] = list(csv.reader(fh, delimiter="\t"))

    gene_dict_i[k] = {g[1]: i for i, g in enumerate(feature_dict[k])}


### Basic stats

We compute a bunch of basic stats, such as nUMIs/cell and nGenes/cell. We'll need some of these for gene selection and clustering so we'll save them for later.

In [ ]:
# collect the shortread files for the PBMC data
npz_files = {k: next((data_path / k).glob(f"**/{k}.npz")) for k in SHORTREAD_KEYS}

In [ ]:
# we'll cache these in a pickle 
stats_file = data_path / "shortread_full_stats.pickle"
if not stats_file.exists():
    numis = {k: sparse.load_npz(npz_files[k]).sum(1).todense() for k in SHORTREAD_KEYS}
    ngenes = {k: np.sign(sparse.load_npz(npz_files[k])).change_compressed_axes((0,)).sum(1).todense() for k in SHORTREAD_KEYS}

    # we will compute these later
    ix_dict = dict()
else:
    # if we've already made the pickle file, this is a lot quicker
    with stats_file.open("rb") as fh:
        d = pickle.load(fh)
        numis = d["numis"]
        ngenes = d["ngenes"]

        # these are computed later
        ix_dict = d["ix_dict"]

In [ ]:
# total nUMIs per experiment--these were not loaded equally on the flowcell
for k in SHORTREAD_KEYS:
    print(f"{k}\tnumis: {numis[k].sum():,}")

In [ ]:
# kneeplots

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for k in SHORTREAD_KEYS:
    ax.plot(sorted(numis[k], reverse=True), label=SHORTREAD_NAMES[k], color=SAMPLE_COLORS[SHORTREAD_NAMES[k]])
    ax.set_xscale("log")
    ax.set_yscale("log")

ax.legend()
ax.set_title("kneeplots for short-read scRNA (full data)", fontsize="medium")

plt.show()


Above we plot kneeplots (barcodes sorted by # UMIs, in descending order) for each of the PBMC datasets. The PIPseq data has lower UMI counts but contains considerably more barcodes above 1000 UMIs.

### Cell selection

We're using a very simple filter here: we select all barcodes >1000 nUMIs. There is additional filtering that could be done (for instance, we are not filtering based on mitochondrial percent) but for our analysis a simple threshold was sufficient.

In [ ]:
if len(ix_dict) == 0:
    for k in SHORTREAD_KEYS:
        ix_dict[k] = numis[k] > 1000

# total nUMIs per experiment--these were not loaded equally on the flowcell
for k in SHORTREAD_KEYS:
    print(k)
    print(f"numis: {numis[k].sum():,}\tbarcodes: {ix_dict[k].sum():,}\t\tnumis in barcodes: {numis[k][ix_dict[k]].sum():,}")

if not stats_file.exists():
    with stats_file.open("wb") as out:
        pickle.dump(
            {
                "numis": numis,
                "ngenes": ngenes,
                "ix_dict": ix_dict,
            },
            out
        )


# Subsampling

Now that we have an estimate of the number of cells in each experiment, we will subsample our BAMs to have roughly equal number of reads per cell.

To skip all of this, download the `subsample_stats.pickle` file and go to **Saturation plots**

In [ ]:
# uncomment to download the subsampling stats only
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/notebook_checkpoints/subsample_stats.pickle {data_path}

# uncomment to download the BAM files. Warning: these are huge
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_pipseq/starsolo_out.bam {data_path}/pipseq_pbmc/starsolo_out.bam
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_10x_3p/outs/possorted_genome_bam.bam {data_path}/10x_3p_pbmc/outs/possorted_genome_bam.bam
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_10x_3p/outs/possorted_genome_bam.bam {data_path}/10x_5p_pbmc/outs/possorted_genome_bam.bam

In [ ]:
subsample_stats = data_path / "subsample_stats.pickle"

sample_bams = {
    "pipseq_pbmc": data_path / "pipseq_pbmc" / "starsolo_out.bam",
    "10x_3p_pbmc": data_path / "10x_3p_pbmc" / "outs" / "possorted_genome_bam.bam",
    "10x_5p_pbmc": data_path / "10x_5p_pbmc" / "outs" / "possorted_genome_bam.bam",
}

sample_bug_npz = {
    "pipseq_pbmc": data_path / "pipseq_pbmc" / "pipseq_pbmc_full.bug_vec.npz",
    "10x_3p_pbmc": data_path / "10x_3p_pbmc" / "10x_3p_pbmc_full.bug_vec.npz",
    "10x_5p_pbmc": data_path / "10x_5p_pbmc" / "10x_5p_pbmc_full.bug_vec.npz",
}

# first element is the tag used for gene assignment
# second element is whether to use 10x's "xf" filter
sample_args = {
    "pipseq_pbmc": ("gx", False),
    "10x_3p_pbmc": ("GX", True),
    "10x_5p_pbmc": ("GX", True),
}

In [ ]:
# calculate the number barcodes and number of genes for a range of UMI thresholds
def calc_stats(m, thresholds):
    barcodes = m.sum(1).todense()[:, None] > thresholds

    n_barcodes = barcodes.sum(0)
    n_genes = np.sign(m).sum(1).todense()
    n_genes = [n_genes[barcodes[:,i]] for i in range(barcodes.shape[1])]
    
    return n_barcodes, n_genes


# returns the nUMI distribution over a given threshold
def numi_dist(m, threshold=1000):
    numis = m.sum(1).todense()
    return numis[numis > threshold]


# called bugzapper and save the output as an npz file
def make_bug_array(input_bam, out_path, barcodes, feature_list, gt, check_xf):
    bug_vec = np.array(
        bugzapper.build_bug_vec(input_bam, barcodes, feature_list, gt, check_xf)
    )
    np.savez(out_path, bug_vec=bug_vec)

In [ ]:
# the output from this cell can be downloaded, to avoid downloading the raw BAM files
for key in SHORTREAD_KEYS:
    out_path = sample_bug_npz[key]
    print(sample_bams[key], out_path.name, sample_args[key], end=" ")
    feature_list = [r[0] for r in feature_dict[key]]
    if not out_path.exists():
        make_bug_array(
            sample_bams[key],
            out_path,
            bc_dict[key],
            feature_list,
            *sample_args[key],
        )
    else:
        print("... already exists")

### Subsampling procedure

The above code takes the BAM file and creates a large but _relatively_ compact array that summarizes the read counts for each UMI. Each row in the array represents one UMI, with indexes into the barcode and feature lists, as well as a read count.

To subsample the data, we can sample the read counts with a binomial distribution corresponding to the depth we want to achieve. Any UMI that has ≥1 read remaining will be counted in the resulting subsample. By creating the array above we can efficiently try many different subsamples.

In [ ]:
# make subsampled bug_vec arrays at 1k, 2k ... 10k, 15k ... 100k reads per cell
# 100k is very slightly higher than the depth of the PIPseq data so we will just use
# the full data there
reads_per_cell = np.hstack(
    [np.arange(1000, 10000, 1000), np.arange(10000, 100001, 5000)]
)

# consider various nUMI thresholds
thresholds = np.array([[100, 200, 500, 1000, 2000, 5000]])

In [ ]:
for key in SHORTREAD_KEYS:
    print(SHORTREAD_NAMES[key], end=" ")
    subsample_p = data_path / key / "subsample"
    subsample_p.mkdir(exist_ok=True)

    if all((subsample_p / f"{key}_{rpc // 1000}k.npz").exists() for rpc in reads_per_cell):
        print("... all files present")
        continue

    full_data = np.load(sample_bug_npz[key])["bug_vec"]
    n_reads = full_data[:, 2].sum()

    for rpc in reads_per_cell:
        pct = ix_dict[key].sum() * rpc / n_reads
        print(f"{rpc}", end=" ")
        output = subsample_p / f"{key}_{rpc // 1000}k.npz"
        if output.exists():
            continue

        if pct < 1.0:
            v = (rng.binomial(full_data[:, 2], pct) > 0).astype(int)
        else:
            v = np.ones(full_data.shape[0], dtype=int)

        m = sparse.COO(
            full_data[:, :2].T,
            data=v,
            shape=(len(bc_dict[key]), len(feature_dict[key])),
            fill_value=0,
            prune=True
        ).asformat("gcxs", compressed_axes=(0,))

        sparse.save_npz(output, m)
    print()

In [ ]:
umi_threshold = 1000

if not subsample_stats.exists():
    # rpc -> threshold -> n
    n_barcodes = defaultdict(lambda: defaultdict(dict))
    # rpc -> threshold -> array
    n_genes = defaultdict(lambda: defaultdict(dict))
    numi_percentiles = defaultdict(list)

    for key in SHORTREAD_KEYS:
        print(SHORTREAD_NAMES[key], end=" ")
        subsample_p = data_path / key / "subsample"
        for rpc in reads_per_cell:
            print(rpc, end=" ")
            m = sparse.load_npz(subsample_p / f"{key}_{rpc // 1000}k.npz")
            ndist = numi_dist(m, umi_threshold)
            if len(ndist):
                numi_percentiles[key].append(np.percentile(ndist, (25, 50, 75)))
            else:
                numi_percentiles[key].append([0.0, 0.0, 0.0])
            n_barcodes[key][rpc], n_genes[key][rpc] = calc_stats(m, thresholds)    
    
        print()

    n_barcodes = {key: dict(d) for key, d in n_barcodes.items()}
    n_genes = {key: dict(d) for key, d in n_genes.items()}
    numi_percentiles = {key: np.vstack(d) for key, d in numi_percentiles.items()}

    with subsample_stats.open("wb") as out:
        pickle.dump((n_barcodes, n_genes, numi_percentiles), out)

else:
    with subsample_stats.open("rb") as fh:
        n_barcodes, n_genes, numi_percentiles = pickle.load(fh)


### Saturation plots

In [ ]:
fig, axs = plt.subplots(
    1, 3, figsize=(18, 6), sharex=True, sharey=True,
    gridspec_kw={"wspace": 0.1, "hspace": 0.1, "bottom": 0.15}
)

for key, ax in zip(SHORTREAD_KEYS, axs.flat):
    ax.plot(
        reads_per_cell, 
        np.vstack([n_barcodes[key][rpc] for rpc in reads_per_cell]),
        label=thresholds.flatten(),
    )

    ax.set_yscale("log")
    ax.set_xlabel("Reads per barcode")
    ax.set_title(SHORTREAD_NAMES[key])

fig.legend(*ax.get_legend_handles_labels(), ncols=6, loc="lower center")
axs[0].set_ylabel("#BC above UMI threshold")
fig.suptitle("Supplemental figure: # barcodes at different subsampling rates")
plt.show()

This plot above shows how the number of barcodes changes with subsampling, using a variety of thresholds. We see that all three of the experiments are quite saturated in terms of the cells--even at 10% subsampling we see almost every barcode. The low-threshold curves reflect that as sequencing continues, empty droplets accumulate more and more UMIs. PIPseq has far more empty droplets and appears to have more ambient mRNA, so these curves are noticeably higher.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

for key in SHORTREAD_KEYS:
    ax.plot(
        reads_per_cell, 
        numi_percentiles[key][:, 1],
        linewidth=2,
        c=SAMPLE_COLORS[SHORTREAD_NAMES[key]],
        label=SHORTREAD_NAMES[key],
    )
    ax.fill_between(
        reads_per_cell,
        numi_percentiles[key][:,0],
        numi_percentiles[key][:,2],
        alpha=0.1,
        color=SAMPLE_COLORS[SHORTREAD_NAMES[key]],
    )

ax.set_xlabel("Reads per barcode")
ax.set_ylabel("#UMIS (barcode > 1000 UMIs)")
ax.legend(loc="upper left")
ax.set_title("UMI distributions for called barcodes")

plt.savefig(figure_path / "supp_fig12.svg")
plt.show()

This plot show the distribution of UMIs/cell as sequencing depth increases. The shaded area is the 50% region around the median. Again these experiments are fairly well saturated, although there is less of a plateau here than in the barcode plot.

## Using subsampled data for metrics and clustering

In [ ]:
# uncomment to download the subsampled npz files, if desired
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_10x_3p/subsample/10x_3p_pbmc_100k.npz {data_path}/10x_3p_pbmc/subsample/
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_10x_5p/subsample/10x_5p_pbmc_100k.npz {data_path}/10x_5p_pbmc/subsample/
! # gcloud storage cp gs://mdl-sc-isoform-2025-ms/pbmc_pipseq/subsample/pipseq_pbmc_100k.npz {data_path}/pipseq_pbmc/subsample/

In [ ]:
subsampled_npz = {
    k: data_path / k / "subsample" / f"{k}_100k.npz" for k in SHORTREAD_KEYS
}

# we'll cache these in a pickle
stats_file = data_path / "shortread_stats_100k.pickle"
if not stats_file.exists():
    subsampled_numis = {
        k: sparse.load_npz(subsampled_npz[k]).sum(1).todense()
        for k in SHORTREAD_KEYS
    }
    subsampled_ngenes = {
        k: np.sign(sparse.load_npz(subsampled_npz[k])).change_compressed_axes((0,)).sum(1).todense()
        for k in SHORTREAD_KEYS
    }

    subsampled_ix = {k: subsampled_numis[k] > 1000 for k in SHORTREAD_KEYS}
    subsampled_g = dict()
else:
    # if we've already made the pickle file, this is a lot quicker
    with stats_file.open("rb") as fh:
        d = pickle.load(fh)
        subsampled_numis = d["numis"]
        subsampled_ngenes = d["ngenes"]

        # these are computed later
        subsampled_ix = d["ix_dict"]
        subsampled_g = d["sel_g"]


In [ ]:
# subsampled nUMIs per experiment
for k in SHORTREAD_KEYS:
    print(k)
    print(f"numis: {subsampled_numis[k].sum():,}\tbarcodes: {subsampled_ix[k].sum():,}\t\tnumis in barcodes: {subsampled_numis[k][subsampled_ix[k]].sum():,}")

### Gene Selection

We're using a fairly simple deviance-from-poisson test for selecting variable genes. The basic idea is to just check if a given gene appears in fewer cells than we'd expect based on its expression--this indicates that it is not homogenously expressed in the population.

In [ ]:
if len(subsampled_g) == 0:
    for k in SHORTREAD_KEYS:
        m = sparse.load_npz(subsampled_npz[k])[subsampled_ix[k], :]
        exp_nz, pct, exp_p = fit_poisson(m)

        subsampled_g[k] = ((exp_nz - pct) > 0.1) & (exp_p < -5)

for k in SHORTREAD_KEYS:
    print(f"selected genes for {k}: {subsampled_g[k].sum():,d}")


In [ ]:
# create the cache file if we don't have one
if not stats_file.exists():
    with stats_file.open("wb") as out:
        pickle.dump(
            {
                "numis": subsampled_numis, 
                "ngenes": subsampled_ngenes,
                "ix_dict": subsampled_ix,
                "sel_g": subsampled_g,
            },
            out
        )


In [ ]:
for k in SHORTREAD_KEYS:
    print(
        k,
        np.median(subsampled_numis[k][subsampled_ix[k]]),
        np.median(subsampled_ngenes[k][subsampled_ix[k]])
    )

### Supplementary figure: metrics for short-read data

Supplementary figure 1 consists of kneeplots for the subsampled data, along with violin plots for UMIs/cell and genes/cell.

In [ ]:
sample_names = [SHORTREAD_NAMES[k] for k in SHORTREAD_KEYS]

# kneeplots
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for key in SHORTREAD_KEYS:
    name = SHORTREAD_NAMES[key]
    ax.plot(
        sorted(subsampled_numis[key], reverse=True), label=name, color=SAMPLE_COLORS[name]
    )
    ax.set_xscale("log")
    ax.set_yscale("log")

ax.set_title("Supplementary figure 1a: kneeplots for short-read scRNA", fontsize="medium")

ax.legend(
    handles=[
        Line2D(markerfacecolor=SAMPLE_COLORS[n], label=n, **line_kwargs)
        for n in sample_names
    ],
    loc="lower left"
)
plt.savefig(figure_path / "supp_fig1a.svg")
plt.show()


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8))

plot_dists(
    ax[0],
    [subsampled_numis[k][subsampled_ix[k]] for k in SHORTREAD_KEYS], 
    log=True,
    colors=[SAMPLE_COLORS[n] for n in sample_names],
    labels=sample_names,
    title="# UMIs per cell",
)
ax[0].set_yticks([3, 4], minor=False)
ax[0].set_yticks(
    np.log10([900, *np.linspace(2000, 9000, 8), *np.linspace(20000, 70000, 6)]),
    minor=True
)

plot_dists(
    ax[1],
    [subsampled_ngenes[k][subsampled_ix[k]] for k in SHORTREAD_KEYS], 
    log=True,
    colors=[SAMPLE_COLORS[n] for n in sample_names],
    labels=sample_names,
    title="# genes per cell",
)
ax[1].set_yticks([2, 3, 4], minor=False)
ax[1].set_yticks(
    np.log10([v*10**i for i in range(1, 4) for v in range(2, 10)][3:]),
    minor=True
)


fig.suptitle("Supplementary figure 1bc: UMI and gene distributions for short-read data")
plt.savefig(figure_path / "supp_fig1bc.svg")
plt.show()


### Graph embedding

We calculate a shared-nearest neighbor graph (using code in `isoscelles`) and then use graphviz to make a graph embedding with the `sfdp` algorithm.

In [ ]:
# again we're going to cache these for easy re-runs
graph_file = data_path / "shortread_graphs_100k.pickle"
graph_file.exists()

In [ ]:
if not graph_file.exists():
    coords = dict()

    for k in SHORTREAD_KEYS:
        print(k)
        exp = sparse.load_npz(subsampled_npz[k])[subsampled_ix[k], :][:, subsampled_g[k]]
        graph = nbrs.calc_graph(np.sqrt(exp))
        coords[k] = layout_graph(graph)

    # the graph embedding has arbitrary orientation, we might need to flip the coordinates
    # around to match cell types up in the different datasets
    coords['10x_3p_pbmc'] *= np.array([[-1, 1]])  # flip 10x 3' x coordinate

    with graph_file.open("wb") as out:
        pickle.dump(coords, out)
else:
    with graph_file.open("rb") as fh:
        coords = pickle.load(fh)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 6), gridspec_kw={"hspace": 0.1, "wspace": 0.1, "bottom": 0.15})
for k, ax in zip(SHORTREAD_KEYS, axs.flat):
    m = ax.scatter(
        coords[k][:, 0], coords[k][:, 1], s=1, c=subsampled_numis[k][subsampled_ix[k]], 
        norm=colors.LogNorm(*np.percentile(subsampled_numis[k][subsampled_ix[k]], (1, 99))),
        rasterized=True,
    )
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    ax.set_title(SHORTREAD_NAMES[k])

    axins1 = inset_axes(
        ax,
        width="20%",
        height="5%",
        loc="lower left",
        bbox_to_anchor=(0.05, 0.05, 0.9, 1),
        bbox_transform=ax.transAxes,
    )
    axins1.xaxis.set_ticks_position("bottom")
    fig.colorbar(m, cax=axins1, orientation="horizontal", ticks=[1000, 10000, 100000])

fig.suptitle("Embeddings of single cell data, colored by UMIs")

plt.show()

### Clustering

The clustering code is in `mdl.isoscelles.leiden`. The basic strategy is:

  1. Select genes using poisson test (as above)
  2. Compute SNN graph
  3. Cluster with Leiden algorithm over increasing range of resolutions until we find more than one cluster
  4. Recursively subcluster the results until no more clusters are found

In [ ]:
clustering_file = data_path / "shortread_clustering_100k.pickle"
clustering_file.exists()

In [ ]:
# create the cache file if we don't have one
if not clustering_file.exists():
    clustering = dict()

    for k in SHORTREAD_KEYS:
        print(k)
        res_list = [float(f"{b}e{p}") for p in range(-7, -2) for b in range(1, 10)]
        data = sparse.load_npz(subsampled_npz[k])[subsampled_ix[k], :]
        clustering[k] = recursive_cluster(data, res_list, feature_cutoff_pct=0.05)
    with clustering_file.open("wb") as out:
        pickle.dump(clustering, out)
else:
    with clustering_file.open("rb") as fh:
        clustering = pickle.load(fh)

for k in SHORTREAD_KEYS:
    print(f"{k}: found {len(cluster_leaf_nodes(clustering[k][0], n=80)):2d} clusters")

In [ ]:
c_arrays = dict()
for key in SHORTREAD_KEYS:
    _leaf_nodes = cluster_leaf_nodes(clustering[key][0])
    _label_array = cluster_labels(clustering[key][0], _leaf_nodes)
    _k2i = {k: i for i, k in enumerate(sorted(_leaf_nodes))}    
    c_arrays[key] = np.array([_k2i.get(k, -1) for k in _label_array])

#### Markers

Here we go through a list of markers for various cell types and plot expression on our embeddings. Note: we need the data files for this, it isn't stored in the pickle.

In [ ]:
def plot_markers(markers):
    _, axs = plt.subplots(
        len(markers), 3, figsize=(9, len(markers) * 3), squeeze=False, gridspec_kw={"hspace": 0.1, "wspace": 0.1}
    )

    for i,k in enumerate(SHORTREAD_KEYS):
        d = sparse.load_npz(subsampled_npz[k])[subsampled_ix[k], :]
        for j,g in enumerate(markers):
            d2 = d[:, gene_dict_i[k][g]].todense()

            axs[j, i].scatter(coords[k][:, 0], coords[k][:, 1], s=1, cmap="Blues", c=np.log1p(d2), alpha=0.8)
            axs[j, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
        axs[0, i].set_title(SHORTREAD_NAMES[k])

    for j,g in enumerate(markers):
        axs[j, 0].set_ylabel(g)

In [ ]:
markers = [
    "CD8A", "CD3D", "CD4", "CD14", "CCR7", "SELL",
    "FCGR3A", "CLEC10A", "CD1C", "NKG7", "NCAM1", "GNLY",
    "MS4A1", "IGKC", "IGLC2", "IRF7", "IGHM", "JCHAIN",
]

plot_markers(markers)

#### Clusters

We annotated the clusters based on the expression of the marker genes above. Here we plot the embedding with clusters labeled.

In [ ]:
cluster_names_colors = {
    "pipseq_pbmc": {
        0: ('CD4 T cells 1', '#006db6'),
        1: ('CD4 T cells 2', '#3d8fc6'),
        2: ('Naive CD4', '#7bb1d6'),
        3: ('Cytotoxic T cells', '#b8d3e5'),
        4: ('Innate Lymphoid', '#80BC42'),
        5: ('CD16 Monocytes', '#888b8d'),
        6: ('CD14 Monocytes', '#4eeebb'),
        7: ('B cells', '#f36c3e'),
    },
    "10x_3p_pbmc": {
        0: ('CD4 T cells 1', '#006db6'),
        1: ('CD4 T cells 2', '#3d8fc6'),
        2: ('Naive CD4', '#7bb1d6'),
        3: ('Cytotoxic T cells', '#b8d3e5'),
        4: ('B cells', '#f36c3e'),
        5: ('CD14 Monocytes', '#4eeebb'),
        6: ('CD16 Monocytes', '#888b8d'),
        7: ('DC', '#0a2240'),
    },
    "10x_5p_pbmc": {
        0: ('CD4 T cells 1', '#006db6'),
        1: ('CD4 T cells 2', '#3d8fc6'),
        2: ('Naive CD4', '#7bb1d6'),
        3: ('Cytotoxic T cells', '#b8d3e5'),
        4: ('Innate Lymphoid', '#80BC42'),
        5: ('B cells', '#f36c3e'),
        6: ('CD14 Monocytes', '#4eeebb'),
        7: ('CD16 Monocytes', '#888b8d'),
    }
}

labels = [
    (cluster_names_colors[k][i][0], SHORTREAD_NAMES[k])
    for k in SHORTREAD_KEYS for i in sorted(cluster_names_colors[k])
]

In [ ]:
# data for supplementary table 1 (when reformatted)
for key in SHORTREAD_KEYS:
    print(key, (c_arrays[key] > -1).sum())
    for i,lbl in sorted(cluster_names_colors[key].items(), key=lambda i: i[1]):
        print(i, f"{(c_arrays[key] == i).sum():5,}", f"{(c_arrays[key] == i).sum() / c_arrays[key].shape[0]:6.1%}", lbl[0], sep="\t")
    print()

In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(20, 6), gridspec_kw={"hspace": 0.1, "wspace": 0.1, "bottom": 0.15})
for k, ax in zip(SHORTREAD_KEYS, axs.flat):
    ix = c_arrays[k] > -1

    # an array of cluster labels for all cells, including the
    # ones that were not fully assigned
    c = np.array([cluster_names_colors[k][i][1] for i in c_arrays[k][ix]])

    ax.scatter(coords[k][ix, 0], coords[k][ix, 1], s=1, c=c, alpha=0.8, rasterized=True)
    ax.set_title(SHORTREAD_NAMES[k])
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

fig.legend(
    handles=[
        Line2D(markerfacecolor=c, label=lbl, **line_kwargs)
        for lbl,c in sorted({lbl_c for key in SHORTREAD_KEYS for lbl_c in cluster_names_colors[key].values()})
    ],
    loc="lower center", ncol=3, fontsize="medium",
)

plt.savefig(figure_path / "fig1a_clustering.svg")
plt.show()

In [ ]:
# index into our arrays for the marker genes 
marker_ix = {tuple(gene_dict_i[k][g] for g in markers) for k in SHORTREAD_KEYS}
assert len(marker_ix) == 1
marker_ix = marker_ix.pop()

label_ix = sorted(range(len(labels)), key=lambda i: labels[i])

In [ ]:
# % nonzero for each gene x cluster
pseudobulk_arrays = dict()
# mean UMIs/cell for each gene x cluster
pseudobulk_count_arrays = dict()

for k in SHORTREAD_KEYS:
    d = sparse.load_npz(subsampled_npz[k])[subsampled_ix[k], :]

    pseudobulk_arrays[k] = np.vstack(
        [
            np.sign(d[c_arrays[k] == i, :]).mean(0).todense()
            for i in np.unique(c_arrays[k]) if i != -1
        ]
    )
    pseudobulk_count_arrays[k] = np.vstack(
        [
            (d[c_arrays[k] == i, :]).mean(0).todense() 
            for i in np.unique(c_arrays[k]) if i != -1
        ]
    )


In [ ]:
# combining the arrays across samples to plot a combined dotplot
marker_nz = np.vstack([pseudobulk_arrays[k][:, marker_ix] for k in SHORTREAD_KEYS]).T
marker_counts = np.vstack([pseudobulk_count_arrays[k][:, marker_ix] for k in SHORTREAD_KEYS]).T

plot_marker_dotplot(
    labels,
    markers,
    marker_nz,
    marker_counts,
    label_ix,
    figure_path / "fig1b_dotplot.svg"
)